In [12]:
from tkinter import *
from tkinter import filedialog

from PIL import ImageDraw, Image, ImageGrab
import numpy as np
from skimage import color
from skimage import io
import os
import io

In [15]:
class Paint(object):
    def __init__(self):
        self.root = Tk()
        self.c = Canvas(self.root, bg='white', width=280, height=280)
        self.image1 = Image.new('RGB', (280, 280), color = 'white')
        self.draw = ImageDraw.Draw(self.image1)
        self.c.grid(row=1, columnspan=6)
        #建立“辨識”按鈕
        self.classify_button = Button(self.root, text='辨識',
                                     command=lambda:self.classify(self.c))
        self.classify_button.grid(row=0, column=0, columnspan=2, sticky='EWNS')
        
        #建立“清畫面”按鈕
        self.clear = Button(self.root, text='清畫面',
                           command=self.clear)
        self.clear.grid(row=0, column=2, columnspan=2, sticky='EWNS')
        
        #建立“存檔”按鈕
        self.savefile = Button(self.root, text='存檔',
                              command=self.savefile)
        self.savefile.grid(row=0, column=4, columnspan=2, sticky='EWNS')
        
        #建立“預測”文字框
        self.prediction_text = Text(self.root, height=2, width=10)
        self.prediction_text.grid(row=2, column=4, columnspan=2)
        
        #定義滑鼠事件處理函數
        self.setup()
        
        self.root.mainloop()
    
    #滑鼠事件處理函數
    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = 15
        self.color = 'black'
    
        #定義包含移動及鬆開滑鼠
        self.c.bind('<B1-Motion>', self.paint)
        self.c.bind('<ButtonRelease-1>', self.reset)
    
    #移動滑鼠 處理函數
    def paint(self, event):
        paint_color = self.color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=self.line_width, fill=paint_color, capstyle=ROUND,
                               smooth=TRUE, splinesteps=36)
            self.draw.line((self.old_x, self.old_y, event.x, event.y),
                           fill='black', width=5)
        self.old_x = event.x
        self.old_y = event.y
    
    #鬆開滑鼠按鈕 處理函數
    def reset(self, event):
        self.old_x, self.old_y = None, None
    
    #"清畫面"的處理函數
    def clear(self):
        self.c.delete("all")
        self.image1 = Image.new('RGB', (280, 280), color='white')
        self.draw = ImageDraw.Draw(self.image1)
        self.prediction_text.delete("1.0", END)
        
    #"存檔"的處理函數
    def savefile(self):
        f = filedialog.asksaveasfilename(defaultextension=".png",
                                        filetypes = [("png file",".png")])
        if f is None:
            return
        self.image1.save(f)
        
    #"辨識"的處理函數
    def classify(self, widget):
        pass

if __name__ == '__main__':
    Paint()

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
import os

In [18]:
def getData():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    img_rows, img_cols = 28, 28
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    
    return X_train/255, y_train, X_test/255, y_test

In [19]:
#訓練模型
def trainModel(X_train, y_train, X_test, y_test):
    batch_size = 64
    epochs = 15

    model = tf.keras.models.Sequential()

    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(10, activation='softmax'))

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1)
    
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    datagen.fit(X_train)
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs,
                        validation_data=datagen.flow(X_test, y_test, batch_size=batch_size), verbose=2,
                        steps_per_epoch=X_train.shape[0]//batch_size)

    model.save('mnist_model.h5')
    return model

In [20]:
#載入模型
def loadModel():
    return tf.keras.models.load_model('mnist_model.h5')

In [21]:
#"辨識"處理函數
def classify(self, widget):
    # self.image1.save('原圖.png')
    img = self.image1.resize((28, 28), ImageGrab.Image.ANTIALIAS).convert('L')
    # img.save('縮小.png')
        
    img = np.array(img)
    # Change pixels to work with our classifier
    img = (255 - img) / 255
        
    img2=Image.fromarray(img) 
    #img2.save('2.png')

    img = np.reshape(img, (1, 28, 28, 1))
        
    # Predict digit
    pred = model.predict([img])
    # Get index with highest probability
    pred = np.argmax(pred)
    #print(pred)
    self.prediction_text.delete("1.0", END)
    self.prediction_text.insert(END, pred)

In [24]:
if __name__ == '__main__':
    # 訓練模型或載入既有的模型
    if(os.path.exists('mnist_model.h5')):
        print('load model ...')
        model = loadModel()
    else:
        print('train model ...')
        X_train, y_train, X_test, y_test = getData()
        model = trainModel(X_train, y_train, X_test, y_test)

    print(model.summary())
    
    # 顯示視窗
    Paint()

load model ...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3,